In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=6c120a547bc46db40d3ee360431d3ae23b1f0f7dbd2dfd5e100c849bf4410f04
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Concatenate

import pandas_ta as ta
import requests

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [ ]:
import requests

def fetch_data(symbol, api_key='rJbxLKBGH8LBfX8zWtgB8FczEIUpkUQ7', base_url='https://financialmodelingprep.com/api/v3'):

    # Construct the endpoint URL
    endpoint = f'{base_url}/historical-price-full/{symbol}?apikey={api_key}'

    # Make a GET request to the endpoint
    response = requests.get(endpoint)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Convert the response to JSON format
        data = response.json()

        # Extract historical data and construct a list of dictionaries
        historical_data = data['historical']
        data_list = [{'Date': item['date'], 'Open': item['open'], 'High': item['high'], 'Low': item['low'],
                      'Close': item['close'], 'AdjClose': item['adjClose'], 'Volume': item['volume']} for item in historical_data]

        # Create DataFrame from the list of dictionaries
        df = pd.DataFrame(data_list)

        # Reverse DataFrame to have the latest date on top
        df = df[::-1].reset_index(drop=True)

        return df
    else:
        print('Failed to retrieve data from the API.')
        return None

In [ ]:
symbol = "TSLA"
df = fetch_data(symbol)
if df is not None:
    print(df)

            Date    Open    High     Low   Close  AdjClose     Volume
0     2019-05-08   16.46   16.71   16.28   16.32     16.32   92646000
1     2019-05-09   16.13   16.25   15.80   16.13     16.13  100671000
2     2019-05-10   15.98   16.13   15.73   15.97     15.97  105124500
3     2019-05-13   15.47   15.50   14.97   15.13     15.13  162522000
4     2019-05-14   15.29   15.63   15.20   15.49     15.49  108786000
...          ...     ...     ...     ...     ...       ...        ...
1254  2024-05-01  182.00  185.86  179.01  179.99    179.99   92829719
1255  2024-05-02  182.86  184.60  176.02  180.01    180.01   89148041
1256  2024-05-03  182.10  184.78  178.42  181.19    181.19   75491539
1257  2024-05-06  183.80  187.56  182.20  184.76    184.76   84390253
1258  2024-05-07  182.40  183.26  177.40  177.73    177.73   69324661

[1259 rows x 7 columns]


In [ ]:
df

,Date,Open,High,Low,Close,AdjClose,Volume
0,2019-05-08,16.46,16.71,16.28,16.32,16.32,92646000
1,2019-05-09,16.13,16.25,15.80,16.13,16.13,100671000
2,2019-05-10,15.98,16.13,15.73,15.97,15.97,105124500
3,2019-05-13,15.47,15.50,14.97,15.13,15.13,162522000
4,2019-05-14,15.29,15.63,15.20,15.49,15.49,108786000
...,...,...,...,...,...,...,...
1254,2024-05-01,182.00,185.86,179.01,179.99,179.99,92829719
1255,2024-05-02,182.86,184.60,176.02,180.01,180.01,89148041
1256,2024-05-03,182.10,184.78,178.42,181.19,181.19,75491539
1257,2024-05-06,183.80,187.56,182.20,184.76,184.76,84390253


In [ ]:
# Function to create technical indicators
def create_technical_indicators(data):
    data['MA10'] = ta.sma(data['Close'], length=10)
    data['MA50'] = ta.sma(data['Close'], length=50)
    data['EMAF'] = ta.ema(data.Close, length=20)
    data['EMAM'] = ta.ema(data.Close, length=100)
    data['EMAS'] = ta.ema(data.Close, length=150)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['MACD'], data['MACD_signal'], _ = ta.macd(data['Close'])
    return data


In [ ]:
# Function to preprocess data
def preprocess_data(data,features, sequence_length):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features])
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(scaled_data[i:i+sequence_length])
        y.append(1 if data['TargetMoving'].values[i+sequence_length] > 0 else 0)
    return np.array(X), np.array(y)

In [ ]:
def build_lstm_model(input_shape, num_features):
    lstm_input = Input(shape=input_shape)
    lstm_layer = LSTM(units=128, return_sequences=True)(lstm_input)
    # lstm_layer = BatchNormalization()(lstm_layer)
    # lstm_layer = Dropout(0.2)(lstm_layer)
    lstm_layer = LSTM(units=64, return_sequences=True)(lstm_layer)
    # lstm_layer = BatchNormalization()(lstm_layer)
    # lstm_layer = Dropout(0.2)(lstm_layer)
    lstm_layer = LSTM(units=32)(lstm_layer)

    # Dense layers for final classification
    output = Dense(units=1, activation='sigmoid')(lstm_layer)

    model = Model(inputs=lstm_input, outputs=output)
    return model

In [ ]:
# Function to evaluate model
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# # Function to train model for a stock symbol
# def train_stock_model(stock_symbol, sequence_length=30, features=None):
#     # Fetch stock data

#     data = fetch_data(stock_symbol)
#     data = create_technical_indicators(data)
#     data['TargetMoving'] = data['AdjClose'].shift(-1) - data['AdjClose']
#     data.dropna(inplace=True)

#     # Define features if not specified
#     if features is None:
#         features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'EMAF', 'EMAM', 'EMAS', 'RSI']

#     # Preprocess data
#     X, y = preprocess_data(data, features, sequence_length)
#     # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     split_ratio = 0.8
#     split_index = int(split_ratio * len(X))

#     X_train = X[:split_index]
#     y_train = y[:split_index]
#     X_test = X[split_index:]
#     y_test = y[split_index:]

#     # Build LSTM model
#     model = build_lstm_model((X_train.shape[1], X_train.shape[2]), len(features))
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#     # Train model
#     model.fit([X_train], y_train,
#               epochs=50, batch_size=64, validation_data=([X_test], y_test), verbose=1)

#     return model

In [ ]:
# # Function to predict tomorrow's movement for a stock symbol
# def predict_tomorrow_movement(stock_symbol, model, sequence_length=30, features=None):
#     # Fetch latest data for the specified stock symbol
#     latest_data = fetch_data(stock_symbol)
#     latest_data = create_technical_indicators(latest_data)

#     # Prepare input data for the model
#     latest_features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'EMAF', 'EMAM', 'EMAS', 'RSI']
#     if features is not None:
#         latest_features = features



#     latest_data = latest_data[latest_features][-sequence_length:]
#     scaler = MinMaxScaler()
#     latest_data = scaler.fit_transform(latest_data[latest_features])
#     latest_data = np.expand_dims(latest_data, axis=0)


#     # Make prediction
#     prediction = model.predict(latest_data)
#     predicted_movement = "Up" if prediction > 0.5 else "Down"

#     return predicted_movement

In [ ]:
# def predict_tomorrow_movement_for_symbols(stock_symbols, sequence_length=30, features=None):
#     predictions = {}
#     for symbol in stock_symbols:
#         model = train_stock_model(symbol)
#         prediction = predict_tomorrow_movement(symbol, model, sequence_length, features)
#         predictions[symbol] = prediction
#     return predictions

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle


def train_stock_model(stock_symbol, sequence_length=30, features=None):
    # Fetch stock data
    data = fetch_data(stock_symbol)
    data = create_technical_indicators(data)
    data['TargetMoving'] = data['AdjClose'].shift(-1) - data['AdjClose']
    data.dropna(inplace=True)

    # Define features if not specified
    if features is None:
        features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'RSI']

    # Preprocess data
    X, y = preprocess_data(data, features, sequence_length)
    X, y = shuffle(X, y, random_state=42)  # Shuffle the data to remove any underlying order
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build LSTM model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]), len(features))     #build_lstm_model(sequence, columns)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train model
    # model.fit([X_train], y_train,
    #           epochs=50, batch_size=64,
    #           validation_data=([X_test], y_test),
    #           callbacks=[early_stopping], verbose=1)

    model.fit([X_train], y_train,
               epochs=50, batch_size=64, validation_data=([X_test], y_test), verbose=1)


    acc = evaluate_model(model, X_test, y_test)
    print("accuracy: ",acc)
    print(stock_symbol)


    return model



In [ ]:
# Function to predict tomorrow's movement for a stock symbol
def predict_tomorrow_movement(stock_symbol, trained_models, sequence_length=30, features=None):
    # Fetch latest data for the specified stock symbol
    data = fetch_data(stock_symbol)
    latest_data = create_technical_indicators(data)
    model = trained_models[stock_symbol]
    # Prepare input data for the model
    latest_features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'RSI','sent']
    if features is not None:
        latest_features = features

    latest_data = latest_data[latest_features][-sequence_length:]
    # print(latest_data)
    scaler = MinMaxScaler()
    latest_data = scaler.fit_transform(latest_data)
    # print(latest_data)
    latest_data = np.expand_dims(latest_data, axis=0)
    # print(latest_data)

    # Make prediction
    prediction = model.predict(latest_data)
    predicted_movement = "Up" if prediction > 0.5 else "Down"

    return predicted_movement,prediction

In [ ]:
def predict_tomorrow_movement_for_symbols(stock_symbols,trained_models,sequence_length=30, features=None):
    predictions = {}
    for symbol in stock_symbols:
        model = trained_models
        prediction,probabilty = predict_tomorrow_movement(symbol, model, sequence_length, features)
        predictions[symbol] = prediction,probabilty[0][0]
    return predictions

In [ ]:
def train_models_for_bulk_stocks(stock_symbols):
    trained_models = {}
    for stock_symbol in stock_symbols:
        model = train_stock_model(stock_symbol)
        trained_models[stock_symbol] = model
    return trained_models


In [ ]:
# Rank stocks based on predictions
def rank_stocks(bulk_predictions):
    ranked_stocks = sorted(bulk_predictions.items(), key=lambda x: x[1], reverse=True)
    return ranked_stocks

In [ ]:
#for testing

UK = [
    "AZN", "LYG", "HSBC",
    # "ARM",
    "UL", "BP", "GSK", "STLA", "RELX", "DEO", "BTI", "NGG", "FERG", "HLN",
    "BCS", "CCEP", "NWG", "WTW", "PUK", "VOD", "APTV", "IHG", "CNHI", "NVT", "SNN", "ROIV", "PSO",
    "LBTYB", "LBTYK", "LBTYA", "JHG", "ATCOL", "CLVT", "ARBKL", "NOMD", "LIVN", "TRMD",
    # "INDV",
    "IMCR",
    "MANU", "AY", "DAVA", "VTEX", "VRNA", "AUTL", "CNTA", "GSM", "BCYC", "BSIG"
]

Australian = [
    "BHP", "CBA", "RIO", "CSL",
    # "NAB",
    "WBC",
    # "ANZ",
    # "FMG",
    "WES",
    # "MQG",
    # "GMG",
    "WDS", "RMD", "TLS",
    # "TCL",
    "WOW",
    # "WTC",
    # "QBE",
    "ALL",
    # "STO",
    "JHX",
    # "REA",
    # "BXB",
    "COL",
    # "COH",
    "SUN", "AMC",
    # "XRO",
    # "NST",
    # "REH",
    "ORG",
    # "CPU",
    # "SCG",
    "IAG",
    # "S32",
    # "SVW",
    "FPH", "MIN", "CAR",
    # "SHL",
    "ASX", "SOL",
    # "RHC",
    # "PLS",
    "TLC", "APA",
    # "SGP",
    "PME", "BSL",
    # "MPL"
]

JAP = [
    "TM", "MUFG", "8035.T", "SONY", "6861.T",
    # "NTT",
    "8058.T", "6501.T", "4063.T", "9983.T",
    "9984.T", "8316.T", "8031.T", "6098.T", "8001.T", "8766.T", "9433.T", "HMC", "4568.T",
    "6902.T", "4519.T", "8411.T", "2914.T", "4502.T", "6367.T", "6503.T", "3382.T", "5108.T",
    "8053.T", "6702.T", "6178.T", "8801.T", "7011.T", "8002.T", "CAJ", "6954.T", "8725.T",
    "6857.T", "6301.T", "4901.T", "4543.T", "8802.T", "9022.T", "7269.T", "8015.T", "4578.T",
    "8750.T", "9020.T", "6752.T", "9613.T", "5401.T"
]


for symbol in JAP:
    data = fetch_data(symbol)
    data = create_technical_indicators(data)
    data['TargetMoving'] = data['AdjClose'].shift(-1) - data['AdjClose']
    data.dropna(inplace=True)
    print(data)
    print(symbol)
    print("________________________________________________________________________________________________________")

    features = ['Open', 'High', 'Low', 'AdjClose', 'MA10', 'MA50', 'RSI']

    # Preprocess data
    X, y = preprocess_data(data, features, sequence_length=30)
    print(X)

            Date    Open    High     Low   Close  AdjClose  Volume     MA10  \
149   2019-12-09  143.13  143.13  141.86  141.89    128.24  103754  141.825   
150   2019-12-10  141.30  141.66  141.00  141.18    127.59  146983  141.689   
151   2019-12-11  141.33  141.83  140.98  141.62    127.99  139505  141.654   
152   2019-12-12  140.85  141.80  140.23  141.60    127.97  150024  141.648   
153   2019-12-13  142.61  142.88  141.78  142.10    128.43   89136  141.837   
...          ...     ...     ...     ...     ...       ...     ...      ...   
1253  2024-04-30  229.94  229.94  227.00  227.31    227.31  334235  229.544   
1254  2024-05-01  228.31  228.84  226.56  226.83    226.83  264613  228.830   
1255  2024-05-02  230.30  231.41  229.37  230.80    230.80  228134  228.726   
1256  2024-05-03  232.06  233.35  231.36  232.87    232.87  165241  229.141   
1257  2024-05-06  233.62  235.68  233.57  235.60    235.60  208217  229.671   

          MA50        EMAF        EMAM        EMAS 

In [ ]:
stock_symbols  = [
  "2080.SR",
  "2081.SR",
  "2082.SR",
  "2083.SR",
  "5110.SR",
  "4330.SR",
  "4331.SR",
  "4332.SR",
  "4333.SR",
  "4334.SR",
  "4335.SR",
  "4336.SR",
  "4337.SR",
  "4338.SR",
  "4340.SR",
  "4342.SR",
  "4344.SR",
  "4345.SR",
  "4346.SR",
  "4347.SR",
  "4348.SR",
  "4349.SR",
  "4020.SR",
  "4090.SR",
  "4100.SR",
  "4150.SR",
  "4220.SR",
  "4230.SR",
  "4250.SR",
  "4300.SR",
  "4310.SR",
  "4320.SR",
  "4321.SR",
  "4322.SR",
#   "4323.SR",
#   "70403.SR",
]

USA = [
    "MSFT", "AAPL", "NVDA", "GOOG", "GOOGL", "AMZN", "META", "LLY", "AVGO", "JPM", "V", "WMT", "XOM", "UNH",
    "MA", "PG", "HD", "ORCL", "COST", "MRK", "CVX", "ABBV", "CRM", "NFLX", "AMD", "KO", "PEP", "GE", "LIN",
    "ADBE", "TMO", "DIS", "MCD", "TMUS", "ABT", "QCOM", "DHR", "AMAT", "INTU", "CMCSA", "UBER", "NOW", "COP",
    "UNP", "NKE", "PM", "MU", "ISRG", "SCHW"
]

UK = [
    "AZN", "LYG", "HSBC", "UL", "BP", "GSK", "STLA", "RELX", "DEO", "BTI", "NGG", "FERG", "HLN",
    "BCS", "CCEP", "NWG", "WTW", "PUK", "VOD", "APTV", "IHG", "CNHI", "NVT", "SNN", "ROIV", "PSO",
    "LBTYB", "LBTYK", "LBTYA", "JHG", "ATCOL", "CLVT", "ARBKL", "NOMD", "LIVN", "TRMD", "IMCR",
    "MANU", "AY", "DAVA", "VTEX", "VRNA", "AUTL", "CNTA", "GSM", "BCYC", "BSIG"
]

Australian = [
    "BHP", "CBA", "RIO", "CSL", "WBC", "WES", "WDS", "RMD", "TLS", "WOW", "ALL", "JHX", "COL", "SUN", "AMC",
    "ORG", "IAG", "FPH", "MIN", "CAR", "ASX", "SOL", "TLC", "APA", "PME", "BSL"
]

JAP = [
    "TM", "MUFG", "8035.T", "SONY", "6861.T",
    # "NTT",
    "8058.T", "6501.T", "4063.T", "9983.T",
    "9984.T", "8316.T", "8031.T", "6098.T", "8001.T", "8766.T", "9433.T", "HMC", "4568.T",
    "6902.T", "4519.T", "8411.T", "2914.T", "4502.T", "6367.T", "6503.T", "3382.T", "5108.T",
    "8053.T", "6702.T", "6178.T", "8801.T", "7011.T", "8002.T", "CAJ", "6954.T", "8725.T",
    "6857.T", "6301.T", "4901.T", "4543.T", "8802.T", "9022.T", "7269.T", "8015.T", "4578.T",
    "8750.T", "9020.T", "6752.T", "9613.T", "5401.T"
]



trained_models = train_models_for_bulk_stocks(JAP)

Streaming output truncated to the last 5000 lines.
14/14 [==============================] - 2s 166ms/step - loss: 0.6936 - accuracy: 0.5214 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 25/50
14/14 [==============================] - 2s 150ms/step - loss: 0.6920 - accuracy: 0.5203 - val_loss: 0.6971 - val_accuracy: 0.4861
Epoch 26/50
14/14 [==============================] - 2s 113ms/step - loss: 0.6919 - accuracy: 0.4983 - val_loss: 0.7018 - val_accuracy: 0.4676
Epoch 27/50
14/14 [==============================] - 2s 115ms/step - loss: 0.6921 - accuracy: 0.5156 - val_loss: 0.7019 - val_accuracy: 0.4815
Epoch 28/50
14/14 [==============================] - 2s 121ms/step - loss: 0.6924 - accuracy: 0.5295 - val_loss: 0.6953 - val_accuracy: 0.4676
Epoch 29/50
14/14 [==============================] - 2s 117ms/step - loss: 0.6925 - accuracy: 0.5064 - val_loss: 0.6972 - val_accuracy: 0.5000
Epoch 30/50
14/14 [==============================] - 2s 110ms/step - loss: 0.6917 - accuracy: 0.5191 - 

In [ ]:
predictions = predict_tomorrow_movement_for_symbols(stock_symbols,trained_models)
predictions_df = pd.DataFrame(predictions)
print(predictions_df)

1/1 [==============================] - 0s 44ms/step
       TSLA      AAPL     GOOGL
0      Down        Up        Up
1  0.462194  0.517086  0.576231


In [ ]:
stock_symbol = 'TSLA'
prediction,probabilty = predict_tomorrow_movement(stock_symbol, trained_models)
print(prediction)

1/1 [==============================] - 0s 40ms/step
Down


In [ ]:
# Rank stocks based on predictions
def rank_stocks(predictions):
    ranked_stocks = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return ranked_stocks

In [ ]:
# Return top 10 stocks as investment opportunities
def get_top_10_investment_opportunities(ranked_stocks):
    return ranked_stocks[:10]

In [ ]:
ranked_stocks = rank_stocks(predictions)
top_10_investment_opportunities = get_top_10_investment_opportunities(ranked_stocks)
print("Top 10 Investment Opportunities:")
for stock_symbol, probability in top_10_investment_opportunities:
    print(f"{stock_symbol}: {probability}")

In [ ]:
for symbol, model in trained_models.items():
    model.save(f'{symbol}_model.h5')

In [ ]:
import os

# Define the directory path
directory = r"trained_models"

# Ensure the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Assuming you have a dictionary named 'trained_models' containing your trained models
for symbol, model in trained_models.items():
    filepath = os.path.join(directory, f"{symbol}_model.h5")
    model.save(filepath)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

# Define the directory path to be downloaded
directory_to_download = "/content/trained_models"

# Define the path where the zip file will be created
zip_file_path = "/content/models.zip"

# Compress the folder into a zip file
shutil.make_archive('/content/models', 'zip', directory_to_download)

# Move the zip file to the specified path
shutil.move("/content/models.zip", zip_file_path)

# Download the zip file
from google.colab import files
files.download(zip_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>